In [1]:
!pip install -q transformers accelerate peft bitsandbytes datasets trl 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 91.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 75.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import re
from pathlib import Path
from datasets import Dataset

PROBLEMS_DIR = "/kaggle/input/fine-tune-dl-btl/problems"
SOLUTIONS_DIR = "/kaggle/input/fine-tune-dl-btl/solution"

# Lấy danh sách số từ tên file Problem{nums}.md
def extract_number(filename):
    match = re.search(r'Problem(\d+)\.md', filename)
    return int(match.group(1)) if match else None

# Tạo mapping số → nội dung
problem_files = [f for f in os.listdir(PROBLEMS_DIR) if f.endswith('.md')]
solution_files = [f for f in os.listdir(SOLUTIONS_DIR) if f.endswith('.py')]

# Tạo dict: num → problem content
problems_dict = {}
for pf in problem_files:
    num = extract_number(pf)
    if num is not None:
        with open(os.path.join(PROBLEMS_DIR, pf), 'r', encoding='utf-8') as f:
            problems_dict[num] = f.read().strip()

# Tạo dict: num → solution content
solutions_dict = {}
for sf in solution_files:
    match = re.search(r'Solution(\d+)\.py', sf)
    if match:
        num = int(match.group(1))
        with open(os.path.join(SOLUTIONS_DIR, sf), 'r', encoding='utf-8') as f:
            solutions_dict[num] = f.read().strip()

# Ghép cặp theo số
paired_data = []
common_nums = sorted(set(problems_dict.keys()) & set(solutions_dict.keys()))
print(f"Tìm thấy {len(common_nums)} cặp Problems/Solution khớp số.")

for num in common_nums:
    paired_data.append({
        "problem": problems_dict[num],
        "solution": solutions_dict[num]
    })

# Tạo Hugging Face Dataset
dataset = Dataset.from_list(paired_data)



dataset

Tìm thấy 313 cặp Problems/Solution khớp số.


Dataset({
    features: ['problem', 'solution'],
    num_rows: 313
})

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
import torch

model_name = "bigcode/starcoder2-3b"

# load tokenizer + model dùng 4-bit)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
)

# bật gradient checkpointing + LoRA
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# warning không ảnh hưởng âu

2025-11-12 12:51:43.873568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762951904.059149      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762951904.102770      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

In [4]:
dataset[0]

{'problem': 'Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.\nYou may assume that each input would have exactly one solution, and you may not use the same element twice.\nYou can return the answer in any order.\n \nExample 1:\n\nInput: nums = [2,7,11,15], target = 9\nOutput: [0,1]\nExplanation: Because nums[0] + nums[1] == 9, we return [0, 1].\n\nExample 2:\n\nInput: nums = [3,2,4], target = 6\nOutput: [1,2]\n\nExample 3:\n\nInput: nums = [3,3], target = 6\nOutput: [0,1]\n\n \nConstraints:\n\n2 <= nums.length <= 104\n-109 <= nums[i] <= 109\n-109 <= target <= 109\nOnly one valid answer exists.\n\n \nFollow-up: Can you come up with an algorithm that is less than O(n2) time complexity?\n\n\nBoilerplate code:\n```python\ndef twoSum(nums, target):\n    ...\n```',
 'solution': 'def twoSum(nums, target):\n    # two point\n    nums_index = [(v, index) for index, v in enumerate(nums)]\n    nums_index.sort()\n    begin, en

In [5]:
# tạo cột "text" từ problem + solution
def build_text(example):
    instruction = (
        "Instruction: Write clean and efficient Python code that correctly solves the problem."
        "The solution should be the most optimal approach in terms of time and space complexity."
        "Do not include testing or extra text.\n"
    )
    return {
        "text": f"### Problem:\n{example['problem']}\n\n{instruction}\n### Solution:\n{example['solution']}\n\n<|end_of_solution|>"
    }


#  map — xóa các cột cũ để chỉ giữ "text"
dataset_text = dataset.map(
    build_text,
    remove_columns=dataset.column_names,  # xóa 'problem', 'solution'
    desc="Creating text column"
)
dataset_text

Creating text column:   0%|          | 0/313 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 313
})

In [6]:

# Trainer
trainer = SFTTrainer(
    model=model,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,   # effective batch=8
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="starcoder2-finetuned",
        optim="paged_adamw_8bit",
        report_to="none"
    ),
    peft_config=peft_config,
    train_dataset=dataset_text
   
)
trainer.train()
trainer.save_model()

Adding EOS to train dataset:   0%|          | 0/313 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/313 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/313 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.069000
20,1.595100
30,1.169200
40,0.907900
50,0.758300
60,0.727700
70,0.742900
80,0.678700
90,0.608300
100,0.665000


In [8]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model = AutoPeftModelForCausalLM.from_pretrained(
    "/kaggle/working/starcoder2-finetuned",  # thư mục chứa adapter
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/starcoder2-finetuned")

In [10]:

prompt = r'''
Given a list of integers `nums` of size `n`, return the sum of the two largest integers.

Example:
Input: `nums = [1, 2, 3, 4, 5]`
Output: `9`

Explanation:
The two largest integers are 4 and 5, so the sum is 9.

Constraints:
- `n < 10000`


Boilerplate code:
```python
def sum_of_two_largest(nums):
    ...
```

'''


In [11]:
formatted_prompt = f"""### Problem:
{prompt.strip()}

"Instruction: Write clean and efficient Python code that correctly solves the problem."
"The solution should be the most optimal approach in terms of time and space complexity."
"Do not include testing or extra text.\n"

### Solution:
"""

In [12]:
from transformers import GenerationConfig

# Tokenize
inputs = tokenizer(
    formatted_prompt,
    return_tensors="pt",
    padding=False,
    truncation=True,
    max_length=2048
).to("cuda")

# Cấu hình sinh text
generation_config = GenerationConfig(
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

# Sinh
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        generation_config=generation_config
    )

# Giải mã
full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Chỉ lấy phần sau "### Solution:"
generated_code = full_output.split("### Solution:")[-1].strip()
generated_code = generated_code.split("\n\n<|end_of_solution|>")[0].strip()

print("=== Generated Code ===")
print(generated_code)

`generation_config` default values have been modified to match model-specific defaults: {'bos_token_id': 0}. If this is not desired, please set these values explicitly.


=== Generated Code ===
def sum_of_two_largest(nums):
    """
    :type nums: List[int]
    :rtype: int
    """
    largest = second_largest = -sys.maxsize
    for num in nums:
        if num > largest:
            second_largest = largest
            largest = num
        elif num > second_largest:
            second_largest = num
    return largest + second_largest


In [7]:
!zip -r /kaggle/working/starcoder2-finetuned.zip /kaggle/working/starcoder2-finetuned


  adding: kaggle/working/starcoder2-finetuned/ (stored 0%)
  adding: kaggle/working/starcoder2-finetuned/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: kaggle/working/starcoder2-finetuned/tokenizer_config.json (deflated 90%)
  adding: kaggle/working/starcoder2-finetuned/training_args.bin (deflated 51%)
  adding: kaggle/working/starcoder2-finetuned/adapter_config.json (deflated 56%)
  adding: kaggle/working/starcoder2-finetuned/merges.txt (deflated 51%)
  adding: kaggle/working/starcoder2-finetuned/README.md (deflated 44%)
  adding: kaggle/working/starcoder2-finetuned/checkpoint-120/ (stored 0%)
  adding: kaggle/working/starcoder2-finetuned/checkpoint-120/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/starcoder2-finetuned/checkpoint-120/tokenizer_config.json (deflated 90%)
  adding: kaggle/working/starcoder2-finetuned/checkpoint-120/training_args.bin (deflated 51%)
  adding: kaggle/working/starcoder2-finetuned/checkpoint-120/adapter_config.json (deflated 56%)
  adding: kaggle/working/starcoder2-finetuned/checkpoint-120/scheduler.pt (deflated 56%)
  adding: kaggle/working/starcoder2-finetuned/c